# Infer Model Parameters for Individuals in Lung Cancer Treatment Group with Medium Erlotinib Dose

In [2]:
import erlotinib as erlo


# Get data
data = erlo.DataLibrary().lung_cancer_medium_erlotinib_dose_group()

# Create scatter plot
fig = erlo.plots.PKTimeSeriesPlot()
fig.add_data(data, id_key='#ID', time_key='TIME in day', biom_key='PLASMA CONCENTRATION in mg/L', dose_key='DOSE in mg')
fig.set_axis_labels(
    time_label=r'$\text{Time in day}$', biom_label=r'$\text{Plasma concentration in mg/L}$', 
    dose_label=r'$\text{Dose in mg}$')

# Show figure
fig.show()

**Figure 1:** To be completed.

## Find MAP estimates for model parameters

In [7]:
import pints


# Define mechanistic model
path = erlo.ModelLibrary().one_compartment_pk_model()
model = erlo.PharmacokineticModel(path)
model.set_administration(compartment='central', direct=False)
model.set_parameter_names(names={
        'central.drug_amount': 'Central comp. drug amount in mg',
        'dose.drug_amount': 'Dose comp. drug amount in mg',
        'central.size': 'Volume of distribution in L',
        'dose.absorption_rate': 'Absorption rate in 1/day',
        'myokit.elimination_rate': 'Elimination rate in 1/day'})

# Create model
problem = erlo.ProblemModellingController(
    data, id_key='#ID', time_key='TIME in day', biom_keys=['PLASMA CONCENTRATION in mg/L'], dose_key='DOSE in mg')
problem.set_mechanistic_model(model)
problem.set_error_model(
    error_models=[pints.ConstantAndMultiplicativeGaussianLogLikelihood])
problem.fix_parameters(name_value_dict=dict({
    'Central comp. drug amount in mg': 0,
    'Dose comp. drug amount in mg': 0,
    'Noise param 2': 1}))

# Create posterior
log_prior_volume = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior_absorption_rate = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior_elimination_rate = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior_sigma_base = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior_sigma_rel = pints.HalfCauchyLogPrior(location=0, scale=3)
problem.set_log_prior(log_priors=[
    log_prior_volume, 
    log_prior_absorption_rate,
    log_prior_elimination_rate,
    log_prior_sigma_base,
    log_prior_sigma_rel])
log_posteriors = problem.get_log_posteriors()

# Find maximum a posteriori probability estimates (MAP)
opt = erlo.OptimisationController(log_posteriors)
map_estimates = opt.run(n_max_iterations=1000, show_id_progress_bar=True)

### Visualise optimisation results

In [8]:
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(map_estimates)

fig.show()